setp1：定义利益相关者，并进行约束条件需求分析
为了分析安大略湖不同利益相关者的需求并构建约束条件，我们可以假设以下利益相关者和他们对水位的需求：

1. **居民**：
   - 约束条件：水位应保持在平均水位±0.5米以避免洪水或枯水期。
   - 理由：这个范围提供了足够的缓冲以应对自然变化，同时减少洪水或枯水对居民生活的影响。

2. **商业航运公司**：
   - 约束条件：水位至少要高于1.5米，以保证船只可以安全通行，但不超过2.5米，以避免导致岸边设施损害。
   - 理由：1.5米的最低限度确保了大多数船只的安全通行，而2.5米的上限可以防止高水位对岸边设施造成的潜在损害。

3. **休闲活动（如垂钓、划船）**：
   - 约束条件：水位应维持在平均水位±0.3米，以促进休闲活动的进行。
   - 理由：较小的波动范围有助于保持良好的水上休闲条件，同时减少对岸边和水生生态的影响。

4. **发电公司**：
   - 约束条件：水位需要维持在能够高效发电的范围内，比如高于1.8米但不超过2.2米。
   - 理由：这个范围确保了有足够的水流量推动发电机，同时避免过高的水位需要频繁使用溢洪道，从而增加管理成本。

5. **环境保护组织**：
   - 约束条件：水位应保持不低于1.4米，以支持湖泊生态系统的健康。
   - 理由：这个最低限度是为了保护湖泊的水生生物栖息地，避免水位过低导致的生态问题。

这些具体的数值设定基于对各利益相关者需求的综合考虑，旨在平衡安大略湖水位管理中的不同利益，同时考虑到实际可操作性和环境保护的需要。在实际应用中，这些数值需要根据详细的水文学研究、环境评估和社会经济分析进行调整，以确保所有方面的利益得到恰当考虑和平衡。


我们已经计算出了五大湖及其相关水道的理想水位，这些理想水位是基于每个来源的历史平均值计算得出的。以下是各水源的理想水位（单位可能因水源而异，例如，湖泊的水位以米为单位，而河流的流量可能以立方米每秒为单位）：

- Lake Superior (苏必利尔湖) - 平均水位：183.35米
- Lake Michigan-Huron (密歇根湖-休伦湖) - 平均水位：176.33米
- Lake St. Clair (圣克莱尔湖) - 平均水位：175.10米
- Lake Erie (伊利湖) - 平均水位：174.28米
- Lake Ontario (安大略湖) - 平均水位：74.83米
- Detroit River (底特律河) - 平均流量：5862.67立方米/秒
- Niagara River at Buffalo (尼亚加拉河在布法罗) - 平均流量：6005.20立方米/秒
- Ottawa River at Carillon (渥太华河在卡里隆) - 平均流量：2061.25立方米/秒
- St. Clair River (圣克莱尔河) - 平均流量：5626.06立方米/秒
- St. Lawrence River at Cornwall (圣劳伦斯河在康沃尔) - 平均流量：7833.36立方米/秒
- St. Mary's River (圣玛丽河) - 平均流量：2168.24立方米/秒

这些理想水位和流量可以作为我们优化模型的基础，旨在最小化所有湖泊水位与其理想水位之间的差异总和。在实际应用中，需要使用专门的优化算法来考虑各种约束条件，并找到满足所有利益相关者需求的最佳水位配置。此外，还需要根据实际情况调整这些理想水位，以确保它们既符合历史数据，也适应当前和未来的环境变化及社会经济需求。

In [31]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from joblib import Parallel, delayed

# 导入数据并初步探索
data = pd.read_csv('data.csv')  # 从 CSV 文件中读取数据
data_info = data.info()  # 获取数据的信息，如列名、非空值数量等
data_head = data.head()  # 查看数据的前几行
sources_unique = data['Source'].unique()  # 获取数据中湖泊的唯一标识

# 计算每个湖泊每个月份的平均水位，以减少重复计算
monthly_means = {
    lake: {month: data[data['Source'] == f'Lake {lake} - Mean Water Level'][month].mean(numeric_only=True) for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']}
    for lake in ideal_levels.keys()
}

# 定义湖泊的理想水位，历史水位的均值
ideal_levels = {
    'Superior': 183.35,
    'Michigan-Huron': 176.33,
    'St. Clair': 175.10, 
    'Erie': 174.28,
    'Ontario': 74.83
}

# 定义目标函数：最小化所有湖泊水位与其理想水位之间的差异总和
def objective_function(levels):
    total_difference = np.sum(np.abs(levels - np.array(list(ideal_levels.values()))))
    return total_difference

# 修改 constraints_function 函数，加入额外的约束条件
def constraints_function(levels, ideal_levels):
    constraints = []
    # 居民安全约束条件：水位应保持在平均水位±0.6米以避免洪水或枯水期
    for i, lake in enumerate(ideal_levels):
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: x[i] - 0.5 - ideal_levels[lake]})
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: ideal_levels[lake] + 0.5 - x[i]})
    # 船舶航运的约束条件：水位至少要高于1.6米，但不超过2.4米
    for i, lake in enumerate(ideal_levels):
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: x[i] - 1.5 - ideal_levels[lake]})
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: ideal_levels[lake] + 2.5 - x[i]})
    # 娱乐约束条件：水位应维持在平均水位±0.2米
    for i, lake in enumerate(ideal_levels):
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: x[i] - 0.3 - ideal_levels[lake]})
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: ideal_levels[lake] + 0.3 - x[i]})
    # 水力发电约束条件：水位需要维持在能够高效发电的范围内，比如高于1.7米但不超过2.3米
    for i, lake in enumerate(ideal_levels):
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: x[i] - 1.8 - ideal_levels[lake]})
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: ideal_levels[lake] + 2.2 - x[i]})
    # 生态环境的约束条件：水位应保持不低于1.5米
    for i, lake in enumerate(ideal_levels):
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, lake=lake: x[i] - 1.4 - ideal_levels[lake]})
    
    # 添加额外的约束条件，如：
    # - 确保每个湖泊的水位变化不超过2米
    # - 确保相邻湖泊的水位差异不超过1米
    # - 确保某些月份湖泊水位不低于特定值等
    
    # 水位变化不超过2米的约束
    for i, lake in enumerate(ideal_levels):
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i: x[i] - (levels[i] - 0.2)})
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i: (levels[i] + 0.2) - x[i]})
    
    # 相邻湖泊的水位差异不超过1米的约束
    adjacent_lakes = [('Superior', 'Michigan-Huron'), ('Michigan-Huron', 'St. Clair'), ('St. Clair', 'Erie'), ('Erie', 'Ontario')]
    for lake1, lake2 in adjacent_lakes:
        i = list(ideal_levels.keys()).index(lake1)
        j = list(ideal_levels.keys()).index(lake2)
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, j=j: x[i] - x[j] - 1})
        constraints.append({'type': 'ineq', 'fun': lambda x, i=i, j=j: x[j] - x[i] - 1})
    
    return constraints

# 优化函数的修改
def optimize_lake_levels(data):
    newdf = []

    for dd in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
        Erie = data[data['Source'] == 'Lake Erie - Mean Water Level'][dd].mean(numeric_only=True)
        Michigan_Huron = data[data['Source'] == 'Lake Michigan and Lake Huron - Mean Water Level'][dd].mean(numeric_only=True)
        Ontario = data[data['Source'] == 'Lake Ontario - Mean Water Level'][dd].mean(numeric_only=True)
        Clair = data[data['Source'] == 'Lake St. Clair - Mean Water Level'][dd].mean(numeric_only=True)
        Superior = data[data['Source'] == 'Lake Superior - Mean Water Level'][dd].mean(numeric_only=True)

        # 初始水位假设
        initial_levels = [Erie, Michigan_Huron, Ontario, Clair, Superior]
        # 求解优化问题
        result = minimize(objective_function, initial_levels, constraints=constraints_function(initial_levels, ideal_levels),
                          options={'disp': False})  # 设置 options 以关闭详细输出

        optimized_levels = result.x
        print(f"\nOptimization results for {dd}:")
        print(f"   Erie: Optimized: {optimized_levels[0]:.2f}, Ideal: {ideal_levels['Erie']:.2f}")
        print(f"   Michigan: Optimized: {optimized_levels[1]:.2f}, Ideal: {ideal_levels['Michigan-Huron']:.2f}")
        print(f"   Ontario: Optimized: {optimized_levels[2]:.2f}, Ideal: {ideal_levels['Ontario']:.2f}")
        print(f"   Clair: Optimized: {optimized_levels[3]:.2f}, Ideal: {ideal_levels['St. Clair']:.2f}")
        print(f"   Superior: Optimized: {optimized_levels[4]:.2f}, Ideal: {ideal_levels['Superior']:.2f}")
        newdf.append([dd, optimized_levels[0], optimized_levels[1], optimized_levels[2], optimized_levels[3],
                      optimized_levels[4]])

    return newdf

# # 调用优化函数并传入数据
optimized_data = optimize_lake_levels(data)

# # 将结果保存到 CSV 文件 'Q1.csv'
pd.DataFrame(optimized_data, columns=['Month', 'Erie', 'Michigan', 'Ontario', 'Clair', 'Superior']).to_csv('Q1.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Source  253 non-null    object 
 1   Year    253 non-null    int64  
 2   Jan     253 non-null    float64
 3   Feb     253 non-null    float64
 4   Mar     253 non-null    float64
 5   Apr     253 non-null    float64
 6   May     253 non-null    float64
 7   Jun     253 non-null    float64
 8   Jul     253 non-null    float64
 9   Aug     253 non-null    float64
 10  Sep     253 non-null    float64
 11  Oct     253 non-null    float64
 12  Nov     253 non-null    float64
 13  Dec     253 non-null    float64
dtypes: float64(12), int64(1), object(1)
memory usage: 27.8+ KB

Optimization results for Jan:
   Erie: Optimized: 174.35, Ideal: 174.28
   Michigan: Optimized: 176.40, Ideal: 176.33
   Ontario: Optimized: 74.86, Ideal: 74.83
   Clair: Optimized: 174.95, Ideal: 175.10
   Superior: Optimized: 183.07, Id

PermissionError: [Errno 13] Permission denied: 'Q1.csv'